Necessary functions to check the sign condition, compute the Gram matrix, unimodular matrix, and $\alpha^2$.

In [1]:
def sign_cond(alpha_0,beta_0,gamma_0):
    s1 = alpha_0 + beta_0 + gamma_0
    s2 = alpha_0*beta_0 + beta_0*gamma_0 + gamma_0*alpha_0
    s3 = alpha_0*beta_0*gamma_0
    return s1, s2, s3


In [2]:
def factor_coeficient_of_alpha2(alpha_0):
    alpha_poly = alpha_0.lift()
    R = alpha_poly.parent()
    rho = R.gen()

    #  m, n, p from m*rho^2 + n*rho + p
    m = alpha_poly.coefficient(2)
    n = alpha_poly.coefficient(1)
    p = alpha_poly.coefficient(0)

    # Factor coefficients
    m_factored = m.factor()
    n_factored = n.factor()
    p_factored = p.factor()

    print("m = ",m_factored)
    print("n = ",n_factored)
    print("p = ",p_factored)


In [3]:
def to_transition_matrix(base_old, base_new, rho):
    #  [1, rho, rho^2]
    def to_vec(expr):
        expr = SR(expr).expand()  
        return vector(SR, [
            expr.subs(rho=0),
            expr.coefficient(rho),
            expr.coefficient(rho^2)
        ])
    
    B = matrix(SR, [to_vec(b) for b in base_old]).transpose()
    Bp = matrix(SR, [to_vec(bn) for bn in base_new]).transpose()

    U_cols = [B.solve_right(col) for col in Bp.columns()]
    U = matrix(SR, U_cols).transpose()
    return U.apply_map(lambda x: x.simplify_rational())

In [4]:
def rewrite_alpha0_and_compute_norm_psi(alpha_0,get_rho_permutations):
    alpha_poly = alpha_0.lift()
    R = alpha_poly.parent()
    rrho = R.gen()

    coeffs = [alpha_poly.coefficient(i) for i in range(alpha_poly.degree() + 1)]

    common_factor = coeffs[0]
    for c in coeffs[1:]:
        common_factor = gcd(common_factor, c)

    print("k = ",common_factor.factor())

    simplified_coeffs = [c // common_factor for c in coeffs]
    psi = sum(simplified_coeffs[i] * rrho^i for i in range(len(simplified_coeffs)))

    print("psi = ",psi)
    rho1, rho2, rho3 = get_rho_permutations(rho)
    psi1 = psi.subs(r=rho1)
    psi2 = psi.subs(r=rho2)
    psi3 = psi.subs(r=rho3)
    norm = psi1 * psi2 * psi3
    return norm

In [5]:
def compute_gram_matrix(df, rho, basis, get_rho_permutations):
    rho1, rho2, rho3 = get_rho_permutations(rho)
    x1, y1, z1 = basis(rho1)
    x2, y2, z2 = basis(rho2)
    x3, y3, z3 = basis(rho3)

    M_alpha = Matrix([[x2^2 - y2^2, x3^2 - y3^2],
                      [y2^2 - z2^2, y3^2 - z3^2]])
    alpha_0 = M_alpha.det()

    M_beta = Matrix([[x3^2 - y3^2, x1^2 - y1^2],
                     [y3^2 - z3^2, y1^2 - z1^2]])
    beta_0 = M_beta.det()

    M_gamma = Matrix([[x1^2 - y1^2, x2^2 - y2^2],
                      [y1^2 - z1^2, y2^2 - z2^2]])
    gamma_0 = M_gamma.det()
    a = alpha_0*x1^2 + beta_0*x2^2 + gamma_0*x3^2       #||Tx||^2 = ||Ty||^2 = ||Tz||^2
    b = alpha_0*x1*y1 + beta_0*x2*y2 + gamma_0*x3*y3    #<Tx,Ty>
    c = alpha_0*z1*x1 + beta_0*z2*x2 + gamma_0*z3*x3    #<Tz,Tx>
    d = alpha_0*y1*z1 + beta_0*y2*z2 + gamma_0*y3*z3    #<Ty,Tz>
    s1 = sign_cond(alpha_0, beta_0, gamma_0)[0]
    s2 = sign_cond(alpha_0, beta_0, gamma_0)[1]
    s3 = sign_cond(alpha_0, beta_0, gamma_0)[2]
    return s1,s2,s3,a,b,c,d,alpha_0



Shank's  simplest cubic fields, good basis   $\left\{\dfrac{1 + \rho + \rho^2}{3},\rho,\rho+\rho^2\right\}$

In [ ]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    1,
    rho,
    (1+rho+rho^2)/3
] # we know that this is an integral basis.
good_basis = [
    (1+rho+rho^2)/3,
    rho,
    rho+rho^2
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df = r^3 - n*r^2 - (n + 3)*r - 1
L.<rho> = K.quotient(df)

def shank_basis(r):
    x = (1 + r + r^2)/3
    y = r
    z = r + r^2
    return x, y, z
def shank_permutation(rho):
    rho1 = rho
    rho2 = -1 / (1 + rho1)
    rho3 = -1/(1+ rho2)
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=shank_basis, get_rho_permutations=shank_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=shank_permutation).lift()).factor())    


transition matrix = 
[ 0  0 -1]
[ 0  1  0]
[ 1  0  3]
determinant =  1

s1 = (8/9) * (n^2 + 3*n + 9) * (n^2 + 9/2*n + 6)
s2 = (7/81) * (n^2 + 3*n + 9) * (n^6 + 9*n^5 + 236/7*n^4 + 429/7*n^3 + 234/7*n^2 - 459/7*n - 81)
s3 = (7/729) * (n^2 + 3*n + 9) * (n^6 + 9*n^5 + 180/7*n^4 + 135/7*n^3 - 108/7*n^2 - 81/7*n - 243/7)

a = (1/9) * (n^2 + 3*n + 3) * (n^2 + 3*n + 9)
b = (-1/27) * (n^2 + 3*n + 9) * (n^2 + 9*n + 9)
c = (1/27) * (n^2 - 3*n - 9) * (n^2 + 3*n + 9)
d = (2/3) * (n^2 + 3*n + 9)

alpha2 = m*rho^2 + n*rho + p where 
m =  (-2/3) * (n^3 + 47/6*n^2 + 49/2*n + 57/2)
n =  (2/3) * (n^4 + 23/3*n^3 + 151/6*n^2 + 69/2*n + 21/2)
p =  (7/9) * (n^4 + 62/7*n^3 + 256/7*n^2 + 543/7*n + 486/7)

k =  1
psi =  (-2/3*n^3 - 47/9*n^2 - 49/3*n - 19)*r^2 + (2/3*n^4 + 46/9*n^3 + 151/9*n^2 + 23*n + 7)*r + 7/9*n^4 + 62/9*n^3 + 256/9*n^2 + 181/3*n + 54
norm psi =  (7/729) * (n^2 + 3*n + 9) * (n^6 + 9*n^5 + 180/7*n^4 + 135/7*n^3 - 108/7*n^2 - 81/7*n - 243/7)


Washington’s cyclic cubic fields, $n$ even, good basis $\left\{\rho,\dfrac{\rho^2-1}{n-1}-\rho,\rho^2\right\}$

In [11]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    1,
    rho,
    (rho^2 - 1)/(n - 1)
] # we know that this is an integral basis.
good_basis = [
    rho,
    (rho^2 - 1)/(n - 1) - rho,
    rho^2
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3 -(n^3-2*n^2+3*n-3)*r^2-n^2*r-1
L.<rho> = K.quotient(df)

def washington_basis(r):
    x = r
    y = (r^2-1)/(n-1)-r 
    z = r^2
    return x, y, z
def washington_permutation(rho):
    rho1 = rho
    rho2 = -(rho+1)/((n^2-n+1)*rho+n)
    rho3 = -(rho2+1)/((n^2-n+1)*rho2+n)
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=washington_basis, get_rho_permutations=washington_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=washington_permutation).lift()).factor()) 


transition matrix = 
[    0     0     1]
[    1    -1     0]
[    0     1 n - 1]
determinant =  1

s1 = (-2) * (n - 1)^2 * (n^2 - 3*n + 3) * (n^2 + 3) * (n^4 - 2*n^3 + 9/2*n^2 - 13/2*n + 9/2)
s2 = (n - 1)^4 * (n^2 - 3*n + 3)^2 * (n^2 + 3)^2 * (n^8 - 4*n^7 + 12*n^6 - 26*n^5 + 42*n^4 - 52*n^3 + 49*n^2 - 30*n + 11)
s3 = (-1) * (n - 1)^6 * (n^2 - 3*n + 3)^2 * (n^2 + 3)^3

a = (-1) * (n - 1)^2 * (n^2 - 3*n + 3) * (n^2 - n + 3) * (n^2 + 3)
b = (-1) * n * (n - 1)^2 * (n^2 - 3*n + 3) * (n^2 + 3)
c = n * (n - 1)^2 * (n^2 - 3*n + 3) * (n^2 + 3)
d = (-1) * n * (n - 1)^2 * (n^2 - 3*n + 3) * (n^2 + 3)

alpha2 = m*rho^2 + n*rho + p where 
m =  (2) * (n - 1)^2 * (n^2 + 3) * (n^3 - 7/2*n^2 + 5*n - 2)
n =  (-2) * (n - 1)^3 * (n^2 + 3) * (n^5 - 9/2*n^4 + 21/2*n^3 - 31/2*n^2 + 16*n - 8)
p =  (-1) * (n - 1)^2 * (n^2 + 3) * (n^6 - 4*n^5 + 10*n^4 - 21*n^3 + 35*n^2 - 32*n + 13)

k =  (n - 1)^2 * (n^2 + 3)
psi =  (2*n^3 - 7*n^2 + 10*n - 4)*r^2 + (-2*n^6 + 11*n^5 - 30*n^4 + 52*n^3 - 63*n^2 + 48*n - 16)*r - n^6

Washington’s cyclic cubic fields, $n$ odd, good basis $\left\{\rho^2,\dfrac{n-2}{2n-2}\rho^2+\dfrac{\rho}{2}+\dfrac{1}{2n-2},\dfrac{\rho^2+\rho}{2}\right\}$

In [12]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    (rho^2 - 1)/(2*n - 2),
    (rho^2 +rho)/2,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    rho^2,
    (n-2)/(2*n-2)*rho^2+1/2*rho+1/(2*n-2),
    (rho^2+rho)/2
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3 -(n^3-2*n^2+3*n-3)*r^2-n^2*r-1
L.<rho> = K.quotient(df)

def washington_basis(r):
    x = r^2
    y = (n-2)/(2*n-2)*r^2 +r/2+1/(2*n-2)
    z = (r^2+r)/2
    return x, y, z
def washington_permutation(rho):
    rho1 = rho
    rho2 = -(rho+1)/((n^2-n+1)*rho+n)
    rho3 = -(rho2+1)/((n^2-n+1)*rho2+n)
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=washington_basis, get_rho_permutations=washington_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=washington_permutation).lift()).factor()) 


transition matrix = 
[ 0 -1  0]
[ 0  1  1]
[ 1  0  0]
determinant =  -1

s1 = (3/8) * (n^2 - 3*n + 3) * (n^2 + 3) * (n^6 - 14/3*n^5 + 21/2*n^4 - 39/2*n^3 + 74/3*n^2 - 95/6*n + 19/6)
s2 = (9/256) * (n^2 - 3*n + 3) * (n^2 + 3)^2 * (n^14 - 37/3*n^13 + 221/3*n^12 - 872/3*n^11 + 7711/9*n^10 - 17963/9*n^9 + 33878/9*n^8 - 5807*n^7 + 65986/9*n^6 - 67409/9*n^5 + 54266/9*n^4 - 32923/9*n^3 + 13826/9*n^2 - 3307/9*n + 79/3)
s3 = (9/4096) * (n^2 - 3*n + 3) * (n^2 + 3)^3 * (n^8 - 37/3*n^7 + 188/3*n^6 - 171*n^5 + 2410/9*n^4 - 2045/9*n^3 + 76*n^2 + 37/3*n - 19/9)

a = (1/16) * (n^2 - 3*n + 3) * (n^2 + 3) * (n^4 - 5*n^3 + 10*n^2 - 11*n + 1)
b = (1/32) * (n^2 - 4*n + 7) * (n^2 - 3*n + 3) * (n^2 - 2*n - 1) * (n^2 + 3)
c = (1/32) * (n^2 - 3*n + 3) * (n^2 + 3) * (n^4 - 8*n^3 + 16*n^2 - 16*n - 1)
d = (1/64) * (n - 1) * (n^2 - 3*n + 3) * (n^2 + 3) * (n^3 - 11*n^2 + 19*n - 1)

alpha2 = m*rho^2 + n*rho + p where 
m =  (-1/8) * (n - 1)^-1 * (n^2 + 3) * (n^6 - 25/2*n^5 + 83/2*n^4 - 127/2*n^3 + 83/2*n^2 - 5*n - 5)

Washington’s cyclic cubic fields, $n$ odd, good basis $\left\{\dfrac{n-2}{2n-2}\rho^2+\dfrac{n-1}{2n-2}\rho+\dfrac{1}{2n-2},-\dfrac{\rho^2-1}{2n-2}+\rho,\dfrac{\rho^2+\rho}{2}\right\}$

In [13]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    (rho^2 - 1)/(2*n - 2),
    (rho^2 +rho)/2,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    (n-2)/(2*n-2)*rho^2+1/2*rho+1/(2*n-2),
    -(rho^2-1)/(2*n-2)+rho,
    (rho^2+rho)/2
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3 -(n^3-2*n^2+3*n-3)*r^2-n^2*r-1
L.<rho> = K.quotient(df)

def washington_basis(r):
    x = (n-2)/(2*n-2)*r^2 +(n-1)/(2*n-2)*r+1/(2*n-2)
    y = -(r^2-1)/(2*n-2)+r
    z = (r^2+r)/2
    return x, y, z
def washington_permutation(rho):
    rho1 = rho
    rho2 = -(rho+1)/((n^2-n+1)*rho+n)
    rho3 = -(rho2+1)/((n^2-n+1)*rho2+n)
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=washington_basis, get_rho_permutations=washington_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=washington_permutation).lift()).factor()) 


transition matrix = 
[-1 -1  0]
[ 1  2  1]
[ 0 -1  0]
determinant =  -1

s1 = (1/8) * (n^2 - 4*n + 7) * (n^2 - 3*n + 3) * (n^2 + 3) * (n^4 - 2*n^3 + 9/2*n^2 - 13/2*n + 9/2)
s2 = (1/256) * (n^2 - 4*n + 7)^2 * (n^2 - 3*n + 3)^2 * (n^2 + 3)^2 * (n^8 - 4*n^7 + 12*n^6 - 26*n^5 + 42*n^4 - 52*n^3 + 49*n^2 - 30*n + 11)
s3 = (1/4096) * (n^2 - 3*n + 3)^2 * (n^2 - 4*n + 7)^3 * (n^2 + 3)^3

a = (1/32) * (n^2 - 4*n + 7) * (n^2 - 3*n + 3) * (n^2 - 2*n + 3) * (n^2 + 3)
b = (1/64) * (n - 3) * (n - 1) * (n^2 - 4*n + 7) * (n^2 - 3*n + 3) * (n^2 + 3)
c = (1/64) * (n - 3) * (n - 1) * (n^2 - 4*n + 7) * (n^2 - 3*n + 3) * (n^2 + 3)
d = (1/64) * (n - 3) * (n - 1) * (n^2 - 4*n + 7) * (n^2 - 3*n + 3) * (n^2 + 3)

alpha2 = m*rho^2 + n*rho + p where 
m =  (-1/8) * (n^2 - 4*n + 7) * (n^2 + 3) * (n^3 - 7/2*n^2 + 5*n - 2)
n =  (1/8) * (n - 1) * (n^2 - 4*n + 7) * (n^2 + 3) * (n^5 - 9/2*n^4 + 21/2*n^3 - 31/2*n^2 + 16*n - 8)
p =  (1/16) * (n^2 - 4*n + 7) * (n^2 + 3) * (n^6 - 4*n^5 + 10*n^4 - 21*n^3 + 35*n^2 - 32*n + 13

Kishi's cyclic cubic fields, $n \equiv 0,2 \pmod{6} \text{ or } n \equiv 4,10 \pmod{18}$, good basis $\left\{\dfrac{n\rho^2+(n^2+n+2)\rho+1}{n^2+1},\rho,\rho^2\right\}$

In [14]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    (3*n^2+n+3)/(n^2+1)*rho^2 + (n^2+n+2)/(n^2+1)*rho + 1/(n^2+1),
    rho,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    (n*rho^2+(n^2+n+2)*rho+1)/(n^2+1),
    rho,
    rho^2
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3-n*(n^2+n+3)*(n^2+2)*r^2-(n^3+2*n^2+3*n+3)*r-1
L.<rho> = K.quotient(df)

def kishi_basis(r):
    x = (n*r^2+(n^2+n+2)*r+1)/(n^2+1)
    y = r
    z = r^2
    return x, y, z
def kishi_permutation(rho):
    rho1 = rho
    rho2 = -(n*rho+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho+(n^2 + n + 1))
    rho3 = -(n*rho2+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho2+(n^2 + n + 1))
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=kishi_basis, get_rho_permutations=kishi_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=kishi_permutation).lift()).factor())


transition matrix = 
[ 1  0  0]
[ 0  1  0]
[-3  0  1]
determinant =  1

s1 = (2) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^10 + 2*n^9 + 9*n^8 + 9*n^7 + 47/2*n^6 + 15/2*n^5 + 47/2*n^4 - 17/2*n^3 + 21/2*n^2 - 10*n + 5/2)
s2 = (n^2 + 1) * (n^2 + 3) * (n^3 + n^2 + 3*n - 1) * (n^4 + n^3 + 4*n^2 + 3) * (n^21 + 4*n^20 + 24*n^19 + 64*n^18 + 212*n^17 + 411*n^16 + 950*n^15 + 1379*n^14 + 2417*n^13 + 2616*n^12 + 3622*n^11 + 2807*n^10 + 3148*n^9 + 1560*n^8 + 1429*n^7 + 291*n^6 + 176*n^5 - 89*n^4 - 105*n^3 - 51*n^2 - 34*n - 8)
s3 = (n^2 + 3) * (n^2 + 1)^2 * (n^3 + n^2 + 3*n - 1)^2 * (n^4 + n^3 + 4*n^2 + 3) * (n^8 + n^7 + 6*n^6 + 2*n^5 + 8*n^4 - 3*n^3 + 2*n^2 - 1)

a = (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^6 + n^5 + 5*n^4 + n^3 + 5*n^2 - 2*n + 1)
b = n * (n^2 + 3)^2 * (n^4 + n^3 + 4*n^2 + 3)
c = (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^5 + n^4 + 3*n^3 - 1)
d = (-1) * (n^2 + 3) * (n^4 + n^3 + 2*n^2 - n + 1) * (n^4 + n^3 + 4*n^2 + 3)

alpha2 = m*rho^2 + n*rho + p where 
m =  (-2) * (n^12 + 2*n^11 +

Kishi's cyclic cubic fields, $n \equiv 3,5 \pmod{6} \text{ or } n \equiv 1,13 \pmod{18}$, good basis $\left\{-\dfrac{ n\rho^2+(2n^2+n+3)\rho+1}{2(n^2+1)}, \dfrac{\rho^2+\rho}{2},\rho\right\}$

In [15]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    (3*n^2+n+3)/(2*(n^2+1))*rho^2 + (n^2+n+2)/(2*(n^2+1))*rho + 1/(2*(n^2+1)),
    (rho^2+rho)/2,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    (-n)/(2*n^2+2)*rho^2 - (2*n^2+n+3)/(2*n^2+2)*rho - 1/(2*n^2+2),
    (rho^2 + rho)/2,
    rho
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3-n*(n^2+n+3)*(n^2+2)*r^2-(n^3+2*n^2+3*n+3)*r-1
L.<rho> = K.quotient(df)

def kishi_basis(r):
    x = -(n*r^2+(2*n^2+n+3)*r+1)/(2*n^2+2)
    y = (r^2+r)/2
    z = r
    return x, y, z
def kishi_permutation(rho):
    rho1 = rho
    rho2 = -(n*rho+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho+(n^2 + n + 1))
    rho3 = -(n*rho2+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho2+(n^2 + n + 1))
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=kishi_basis, get_rho_permutations=kishi_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=kishi_permutation).lift()).factor())


transition matrix = 
[-1  0  0]
[-1  1  2]
[ 2  0 -1]
determinant =  1

s1 = (-1/4) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^10 + 3/2*n^9 + 15/2*n^8 + 4*n^7 + 59/4*n^6 - 23/4*n^5 + 19/4*n^4 - 101/4*n^3 - 53/4*n^2 - 43/2*n - 35/4)
s2 = (3/256) * n * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^25 + 7/3*n^24 + 56/3*n^23 + 18*n^22 + 287/3*n^21 - 358/3*n^20 - 111*n^19 - 5825/3*n^18 - 8416/3*n^17 - 29198/3*n^16 - 33635/3*n^15 - 74456/3*n^14 - 19907*n^13 - 97199/3*n^12 - 30989/3*n^11 - 32849/3*n^10 + 21510*n^9 + 27807*n^8 + 43201*n^7 + 42060*n^6 + 93494/3*n^5 + 70079/3*n^4 + 8814*n^3 + 12464/3*n^2 + 1175/3*n - 320)
s3 = (-9/4096) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^18 - 5/3*n^17 + 9*n^16 - 307/9*n^15 + 392/9*n^14 - 1751/9*n^13 + 2408/9*n^12 - 3931/9*n^11 + 9794/9*n^10 - 4205/9*n^9 + 17954/9*n^8 - 1339/3*n^7 + 9664/9*n^6 - 6181/9*n^5 - 11584/9*n^4 - 4417/9*n^3 - 4145/3*n^2 - 200*n + 125)

a = (-1/16) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^6 - n^5 + 3*n^4 - 9*n^3 + n^2 - 10*n - 5)
b =

Kishi's cyclic cubic fields, $n \equiv 3,5 \pmod{6} \text{ or } n \equiv 1,13 \pmod{18}$, good basis $\left\{\dfrac{(n^2 - n + 1)\rho^2-(n^2+n+2)\rho-1}{2(n^2+1)}, \dfrac{\rho^2+\rho}{2},-\rho^2\right\}$

In [16]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    (3*n^2+n+3)/(2*(n^2+1))*rho^2 + (n^2+n+2)/(2*(n^2+1))*rho + 1/(2*(n^2+1)),
    (rho^2+rho)/2,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    ((n^2-n+1)*rho^2-(n^2+n+2)*rho-1)/(2*n^2+2),
    (rho^2 + rho)/2,
    -rho^2
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3-n*(n^2+n+3)*(n^2+2)*r^2-(n^3+2*n^2+3*n+3)*r-1
L.<rho> = K.quotient(df)

def kishi_basis(r):
    x = ((n^2-n+1)*r^2-(n^2+n+2)*r-1)/(2*n^2+2)
    y = (r^2+r)/2
    z = -r^2
    return x, y, z
def kishi_permutation(rho):
    rho1 = rho
    rho2 = -(n*rho+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho+(n^2 + n + 1))
    rho3 = -(n*rho2+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho2+(n^2 + n + 1))
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=kishi_basis, get_rho_permutations=kishi_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=kishi_permutation).lift()).factor())


transition matrix = 
[-1  0  0]
[ 0  1  0]
[ 2  0 -1]
determinant =  1

s1 = (3/8) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^10 + 7/3*n^9 + 28/3*n^8 + 26/3*n^7 + 101/6*n^6 - 55/6*n^5 - 27/2*n^4 - 301/6*n^3 - 245/6*n^2 - 35*n - 109/6)
s2 = (9/256) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^26 + 17/3*n^25 + 34*n^24 + 346/3*n^23 + 3443/9*n^22 + 7970/9*n^21 + 17731/9*n^20 + 27943/9*n^19 + 41080/9*n^18 + 3522*n^17 + 6523/9*n^16 - 82732/9*n^15 - 183119/9*n^14 - 339619/9*n^13 - 46905*n^12 - 164279/3*n^11 - 419608/9*n^10 - 337279/9*n^9 - 180245/9*n^8 - 85396/9*n^7 - 932*n^6 + 12919/9*n^5 + 12146/9*n^4 + 8624/9*n^3 + 815/9*n^2 + 268/3*n - 4)
s3 = (9/4096) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^18 + 11/3*n^17 + 47/3*n^16 + 83/3*n^15 + 416/9*n^14 - 47/9*n^13 - 944/9*n^12 - 2855/9*n^11 - 3398/9*n^10 - 1981/9*n^9 + 306*n^8 + 6691/9*n^7 + 2392/3*n^6 + 3331/9*n^5 - 680/9*n^4 - 1397/9*n^3 - 121/3*n^2 + 8*n + 1)

a = (1/16) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^6 + n^5 + 3*n^4 - n^3 - 3*n^2 -

Kishi's cyclic cubic fields, $n \equiv 3,5 \pmod{6} \text{ or } n \equiv 1,13 \pmod{18}$, good basis $\left\{\dfrac{(n^2 + n + 1)\rho^2+(n^2 + n + 2)\rho+1}{2(n^2+1)}, \dfrac{\rho^2+\rho}{2},\rho^2\right\}$

In [ ]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    (3*n^2+n+3)/(2*(n^2+1))*rho^2 + (n^2+n+2)/(2*(n^2+1))*rho + 1/(2*(n^2+1)),
    (rho^2+rho)/2,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    ((n^2+n+1)*rho^2+(n^2+n+2)*rho+1)/(2*n^2+2),
    (rho^2 + rho)/2,
    rho^2
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3-n*(n^2+n+3)*(n^2+2)*r^2-(n^3+2*n^2+3*n+3)*r-1
L.<rho> = K.quotient(df)

def kishi_basis(r):
    x = ((n^2+n+1)*r^2+(n^2+n+2)*r+1)/(2*n^2+2)
    y = (r^2+r)/2
    z = r^2
    return x, y, z
def kishi_permutation(rho):
    rho1 = rho
    rho2 = -(n*rho+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho+(n^2 + n + 1))
    rho3 = -(n*rho2+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho2+(n^2 + n + 1))
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=kishi_basis, get_rho_permutations=kishi_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=kishi_permutation).lift()).factor())


transition matrix = 
[ 1  0  0]
[ 0  1  0]
[-1  0  1]
determinant =  1

s1 = (3/8) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^10 + 5/3*n^9 + 22/3*n^8 + 16/3*n^7 + 101/6*n^6 + 7/2*n^5 + 143/6*n^4 + 23/6*n^3 + 175/6*n^2 + 13/3*n + 95/6)
s2 = (9/256) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^26 + 13/3*n^25 + 82/3*n^24 + 238/3*n^23 + 2531/9*n^22 + 1826/3*n^21 + 13907/9*n^20 + 23827/9*n^19 + 47720/9*n^18 + 7582*n^17 + 12683*n^16 + 145556/9*n^15 + 209989/9*n^14 + 28469*n^13 + 35263*n^12 + 381427/9*n^11 + 129640/3*n^10 + 445981/9*n^9 + 39055*n^8 + 40468*n^7 + 203392/9*n^6 + 60949/3*n^5 + 20630/3*n^4 + 15736/3*n^3 + 6407/9*n^2 + 1348/3*n - 4)
s3 = (9/4096) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^18 + 7/3*n^17 + 35/3*n^16 + 43/3*n^15 + 368/9*n^14 + 209/9*n^13 + 188/3*n^12 + 193/9*n^11 + 458/9*n^10 + 491/9*n^9 - 290/9*n^8 + 329/3*n^7 - 1352/9*n^6 - 509/9*n^5 - 2684/9*n^4 - 2845/9*n^3 - 553/3*n^2 - 308*n - 71)

a = (1/16) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^6 + n^5 + 3*n^4 - n^3 + n^2 

Kishi's cyclic cubic fields, $n \equiv 3,5 \pmod{6} \text{ or } n \equiv 1,13 \pmod{18}$, good basis $\left\{\dfrac{n\rho^2+(n+1)\rho+1}{2(n^2+1)}, \dfrac{\rho^2+\rho}{2},-\rho\right\}$

In [20]:
# We first compute the transition matrix to check if it is unimodular.
n, rho = var('n rho')
basis_old = [
    (3*n^2+n+3)/(2*(n^2+1))*rho^2 + (n^2+n+2)/(2*(n^2+1))*rho + 1/(2*(n^2+1)),
    (rho^2+rho)/2,
    rho^2
] # we know that this is an integral basis.
good_basis = [
    (n*rho^2+(n+1)*rho+1)/(2*n^2+2),
    (rho^2 + rho)/2,
    -rho
]

U = to_transition_matrix(basis_old, good_basis, rho)
print('transition matrix = ')
print(U)
print('determinant = ',det(U))
print()

###
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3-n*(n^2+n+3)*(n^2+2)*r^2-(n^3+2*n^2+3*n+3)*r-1
L.<rho> = K.quotient(df)

def kishi_basis(r):
    x = (n*r^2+(n+1)*r+1)/(2*n^2+2)
    y = (r^2+r)/2
    z = -r
    return x, y, z
def kishi_permutation(rho):
    rho1 = rho
    rho2 = -(n*rho+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho+(n^2 + n + 1))
    rho3 = -(n*rho2+1)/((n^4 + n^3 + 3*n^2 + n + 1)*rho2+(n^2 + n + 1))
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d,alpha_0 = compute_gram_matrix(df, rho, basis=kishi_basis, get_rho_permutations=kishi_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print()
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())
print()
print('alpha2 = m*rho^2 + n*rho + p where ')
factor_coeficient_of_alpha2(alpha_0)
print()
print('norm psi = ',R(rewrite_alpha0_and_compute_norm_psi(alpha_0=alpha_0, get_rho_permutations=kishi_permutation).lift()).factor())


transition matrix = 
[ 1  0  0]
[-1  1 -2]
[-1  0  1]
determinant =  1

s1 = (-1/4) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^10 + 7/2*n^9 + 23/2*n^8 + 23*n^7 + 147/4*n^6 + 201/4*n^5 + 191/4*n^4 + 195/4*n^3 + 115/4*n^2 + 37/2*n + 21/4)
s2 = (3/256) * n * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^25 + 23/3*n^24 + 136/3*n^23 + 190*n^22 + 661*n^21 + 5806/3*n^20 + 4881*n^19 + 32875/3*n^18 + 21632*n^17 + 117062/3*n^16 + 62623*n^15 + 279124/3*n^14 + 123757*n^13 + 458897/3*n^12 + 169641*n^11 + 521671/3*n^10 + 479266/3*n^9 + 400541/3*n^8 + 99281*n^7 + 195296/3*n^6 + 110446/3*n^5 + 51943/3*n^4 + 18274/3*n^3 + 1536*n^2 - 97/3*n - 96)
s3 = (-9/4096) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^18 + 23/3*n^17 + 37*n^16 + 1189/9*n^15 + 3272/9*n^14 + 7529/9*n^13 + 4820/3*n^12 + 8015/3*n^11 + 3906*n^10 + 14921/3*n^9 + 51406/9*n^8 + 50423/9*n^7 + 43904/9*n^6 + 32443/9*n^5 + 6644/3*n^4 + 9847/9*n^3 + 309*n^2 - 36*n - 27)

a = (-1/16) * (n^2 + 3) * (n^4 + n^3 + 4*n^2 + 3) * (n^6 + 3*n^5 + 7*n^4 + 11*n^3 +